In [1]:
#imports
%load_ext autoreload
%autoreload 2
import schillinger.harmony as SH
import numpy as np
import itertools
import random

In [2]:
import schillinger.pitch as SP

In [3]:
from music21 import *
%load_ext music21.ipython21

In [4]:
# GIANT STEPS
harmonic_roots = ["C","E-","F#","A"]
expansions = SP.expansions(harmonic_roots)
print(expansions)
C3 = expansions[0]
C_3 = expansions[1]
#C5 = expansions[2]

root_cycles = SP.pitch_cicles(harmonic_roots,[C3,C_3],[2,1,2,4])# giant steps
#root_cycles = SP.pitch_cicles(harmonic_roots,[C3,C_3,C5],[3,1,2,2,2,3])
print(root_cycles) # All
print(root_cycles[0:10])# Giant Steps

[['C', 'E-', 'F#', 'A'], ['C', 'F#', 'E-', 'A'], ['C', 'A', 'F#', 'E-']]
['C', 'E-', 'F#', 'E-', 'F#', 'A', 'C', 'F#', 'E-', 'A', 'C', 'E-', 'A', 'C', 'E-', 'A', 'C', 'F#', 'E-', 'F#', 'A', 'C', 'E-', 'F#', 'E-', 'A', 'C', 'F#', 'A', 'C', 'F#', 'A', 'C', 'F#', 'E-', 'A', 'C']
['C', 'E-', 'F#', 'E-', 'F#', 'A', 'C', 'F#', 'E-', 'A']


In [5]:
#harmonic_roots_to_individual_harmonies
scale_type = scale.MinorScale
all_root_scales = []
all_expansions = []
for i in harmonic_roots:
    my_temp_scale = scale_type(i)
    all_root_scales.append(my_temp_scale)
    scale_to_pitchnames = [str(p.name) for p in my_temp_scale.pitches[:-1]] # convert to pitchnames
    all_expansions.append(SP.expansions(scale_to_pitchnames))
all_root_scales, all_expansions;

In [6]:
#root_sequence = root_cycles[0:11]
root_sequence = root_cycles
durations = [2,2,2,2, 4,2,2, 2,2,2,2, 4,2,2, 4,2,2, 4,2,2, 4,2,2, 4,2,2] # hardcode
form =      [1,5,1,5, 1,2,5, 1,5,1,5, 1,2,5, 1,2,5, 1,2,5, 1,2,5, 1,2,5]
rythmic_seq = [2]#obsolete

In [7]:
SPG = SP.PitchGroup()

In [8]:
#CADENCE
cadences = []
for exp in all_expansions:
    cadence = SH.diatonic_cadences(exp)
    cadences.append(cadence)
    #print(cadence)
def generate_cadence(idx):
    
    test_cadence = cadences[idx]
    a,b,c,d = test_cadence[0][0],random.choice(test_cadence)[1],random.choice(test_cadence)[2], test_cadence[0][3]
    #print(a,b,c,d)
    return [b,c]

In [32]:
#new
filled_harmonies = []
voice_amount = 4
form_idx = 0
for i, rs in enumerate(root_sequence):
    #print(rs)
    #fill in the first or tonic note
    current_harmony = root_sequence[(i)%len(root_sequence)]
    current_harmony_idx = harmonic_roots.index(current_harmony)
    
    cadence = generate_cadence(current_harmony_idx)
    
    my_scale = all_root_scales[current_harmony_idx]
    expansions = all_expansions[current_harmony_idx] # so
    expansion = expansions[1]
    
    chorified_scale = SPG.chordify_scale(expansion, voice_amount)
    
    #harmonized_bass = SPG.harmonize_bass([rs], chorified_scale)
    
    while form[form_idx%len(form)] != 1:
        
        if form[form_idx%len(form)] == 5:
            
            #change = chorified_scale[2] # DEFAULT
            change = SPG.harmonize_bass([cadence[1]], chorified_scale)[0] #RANDOM CADENCE
            
        if form[form_idx%len(form)] == 2:
            #change = chorified_scale[4] # DEFAULT
            change = SPG.harmonize_bass([cadence[0]], chorified_scale)[0] #RANDOM CADENCE
            
        
        for i in range(durations[form_idx%len(durations)]):
            filled_harmonies.append(change)
        
        form_idx = form_idx + 1
        
    change = chorified_scale[0]
    for i in range(durations[form_idx%len(durations)]):
        filled_harmonies.append(change)
    form_idx = form_idx + 1
    

In [26]:
def showNotes(note_sequece, time_meter, duration_divider):
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    s = stream.Score(id='mainScore')
    p0 = stream.Part(id='part0')
    p0.timeSignature = meter.TimeSignature(TS)
    p1 = stream.Part(id='part1')
    p1.timeSignature = meter.TimeSignature(TS)
    for i, notes in enumerate(note_sequece):
        
        #for y in range():
        dur = rythmic_seq[i%len(rythmic_seq)]
        dur = 1 * duration_divider
        #print(notes)
        c = chord.Chord(notes)
        
        c.quarterLength = dur * duration_divider
        
        c.pitches[2].octave = 3
        c.pitches[0].octave = 2
        #c.inversion(0)
        p0.append(c)
        
        ## fake melody
        melody = []
        
        for i in range(dur):
            nota = pitch.Pitch(random.choice(notes))
            nota.octave = 5
            nota = note.Note(nota.nameWithOctave)
            nota.duration.quarterLength = 1 * duration_divider
            melody.append(nota)   
        
        p1.append(melody)
            
    s.insert(0, p1)
    s.insert(0, p0)
    return s

In [27]:
'''melody = []
for i, m in enumerate(filled_harmonies):
    #print(m)
    melody.append(random.choice(m))
    if i == len(filled_harmonies)-1:
        melody[-1] = m[0]
#melody''';

In [28]:
s = showNotes(filled_harmonies,[4,4],1)
s.show('musicxml')

In [29]:
#return_value = [list(itertools.chain(*filled_harmonies))][0]
#print(return_value)

In [33]:
#pass to another notebooks!
cleaned_filled_harm = list(itertools.chain(*filled_harmonies))
%store cleaned_filled_harm
#del cleaned_filled_harm # This has deleted the variable

Stored 'cleaned_filled_harm' (list)
